Fd find is from <https://github.com/sharkdp/fd> 

In [ ]:
import-module classExplorer
Import-Module Ninmonkey.Console -ea silentlycontinue *>$Null

gcm -m ClassExplorer | ft -auto


CommandType Name                   Version Source
----------- ----                   ------- ------
Cmdlet      Find-Member            2.3.3   classExplorer
Cmdlet      Find-Type              2.3.3   classExplorer
Cmdlet      Format-MemberSignature 2.3.3   classExplorer
Cmdlet      Get-Assembly           2.3.3   classExplorer
Cmdlet      Get-Parameter          2.3.3   classExplorer



Mandatory $PROFILE config
```ps1
# Ensures it's non-bom-utf8 otherwise the static method includes BOM by default
$OutputEncoding = [console]::InputEncoding = [console]::OutputEncoding = [System.Text.UTF8Encoding]::new()
# for colors
$Env:LESS =  '-R'
$Env:Pager = 'Bat'  # less, or bat, either are great. less is on 'choco'
$Env:Pager = 'Less' # less, or bat, either are great. less is on 'choco'

# if you don't want automatic color mode. this ensures more colors, and forces escapes to always use the 24bit color format
[PoshCode.Pansies.RgbColor]::ColorMode = [PoshCode.Pansies.ColorMode]::Rgb24Bit 
```
If you have `Git` installed on windows, you already have a ton of linux utilities, like `less`, `tail`, `printf`, `ls`, `find`, etc . IIRC the `less` from `choco` had the better version of `less`

```ps1
if (Test-Path 'C:\Program Files\Git\usr\bin') {
    $Env:Path += ';', 'C:\Program Files\Git\usr\bin'
}
```

Extra config, 
```ps1
$eaIgnore = @{ ErrorAction = 'ignore' }
$Env:BAT_CONFIG_PATH     = Join-Path $Env:Nin_Dotfiles '/cli/bat/.batrc' | Get-Item @eaIgnore
$Env:PYTHONSTARTUP       = Join-Path $Env:Nin_Dotfiles '/cli/python/nin-py3-x-profile.py' | Get-Item @eaIgnore
$Env:RIPGREP_CONFIG_PATH = Join-Path $Env:Nin_Dotfiles '/cli/ripgrep/.ripgreprc' | Get-Item @eaIgnore
```
if `fzf`
```ps1
$Env:FZF_DEFAULT_COMMAND = 'fd --type file --hidden --exclude .git --color=always'
$Env:FZF_DEFAULT_OPTS = '--ansi --no-height'
```
`StripAnsi` (for below) is at [StripAnsi\(\)](https://github.com/ninmonkey/dotfiles_git/blob/efd8bbccc2e308e53e341808b657ade0e04420ff/powershell/EveryoneShouldImportThese.ps1#L99)

`fd` outputs a lot of color using `ansi escapes`. 
[windows terminal `wt`](https://docs.microsoft.com/en-us/windows/terminal/)
- fully supports utf8
- 24bit color
- There's `$PSStyle` and [Rgb.Pansies](https://github.com/PoshCode/Pansies) to generate the actual escapes for you

In [ ]:
$C = @{ 
    FgDim   = "$fg:gray40"
    Fg      = "$fg:gray70"
    Special = "$fg:green"
    FgBold  = "$fg:gray90"
}
@(
    $C.FgDim
    'Collections.Generic.'
    $C.FgBold
    'List'
    $C.Fg
    '['
    $C.Special
    'object'
    $C.Fg
    ']'
    "$Fg:clear"
) -join ''

Collections.Generic.List[object]


In [ ]:
$cachedLs ??= fd --search-path (gi ~) --color=always -d1 -tf
$cachedLs | select -first 20

C:\Users\cppmo_000\1
C:\Users\cppmo_000\Namespace_AngularSharp.csv
C:\Users\cppmo_000\Today.md
C:\Users\cppmo_000\Untitled-1.jsonc
C:\Users\cppmo_000\_lesshst
C:\Users\cppmo_000\debug.log
C:\Users\cppmo_000\disabled.dir_colors
C:\Users\cppmo_000\foo.sql
C:\Users\cppmo_000\g1016.png
C:\Users\cppmo_000\global.nin.json
C:\Users\cppmo_000\last.txt
C:\Users\cppmo_000\lastast
C:\Users\cppmo_000\mini-error.ps1
C:\Users\cppmo_000\package-lock.json
C:\Users\cppmo_000\prof-2f77
C:\Users\cppmo_000\prof-e748.extHost.cpuprofile.txt
C:\Users\cppmo_000\prof-e748.main.cpuprofile.txt
C:\Users\cppmo_000\prof-e748.renderer.cpuprofile.txt
C:\Users\cppmo_000\pssession.log
C:\Users\cppmo_000\raw text and csv 2021-04-20-export.csv


In [ ]:
$cachedLs | sort { $_.Length }

C:\Users\cppmo_000\1
C:\Users\cppmo_000\sdfds
C:\Users\cppmo_000\lastast
C:\Users\cppmo_000\_lesshst
C:\Users\cppmo_000\prof-2f77
C:\Users\cppmo_000\z-location.db
C:\Users\cppmo_000\Untitled-1.jsonc
C:\Users\cppmo_000\disabled.dir_colors
C:\Users\cppmo_000\foo.sql
C:\Users\cppmo_000\test.txt
C:\Users\cppmo_000\last.txt
C:\Users\cppmo_000\Today.md
C:\Users\cppmo_000\g1016.png
C:\Users\cppmo_000\temp.json
C:\Users\cppmo_000\debug.log
C:\Users\cppmo_000\today.4.md
C:\Users\cppmo_000\pssession.log
C:\Users\cppmo_000\mini-error.ps1
C:\Users\cppmo_000\global.nin.json
C:\Users\cppmo_000\package-lock.json
C:\Users\cppmo_000\Namespace_AngularSharp.csv
C:\Users\cppmo_000\prof-e748.main.cpuprofile.txt
C:\Users\cppmo_000\prof-e748.extHost.cpuprofile.txt
C:\Users\cppmo_000\prof-e748.renderer.cpuprofile.txt
C:\Users\cppmo_000\raw text and csv 2021-04-20-export.csv


A bunch of commands, like: `Sort-Object`, `Where-Object`, `Group-Object`
allow you to benefit by both **colors** (ie: raw text) and **objects** 

Here the `GroupBy` `ScriptBlock` is

- taking the raw text is piped from `fd --color=always`
- stripping ansi escapes
- Converting it to a `[IO.FileSystemInfo]`
- sorting on the `LastWriteTime` member
- extracting the member `Extension` and the comparison key
- finally it pipes the raw ansi escapes

In [ ]:
$CachedLs
| Group { $_ | StripAnsi | gi | sort LastWriteTime | % Extension }
| ft -auto


Count Name        Group
----- ----        -----
    5             {C:\Users\cppmo_000\1, C:\Users\cppmo_000\_lesshst, …
    2 .csv        {C:\Users\cppmo_000\Namespace_AngularSharp.csv, [3…
    1 .db         {C:\Users\cppmo_000\z-location.db}
    1 .dir_colors {C:\Users\cppmo_000\disabled.dir_colors}
    3 .json       {C:\Users\cppmo_000\global.nin.json, C:\U…
    1 .jsonc      {C:\Users\cppmo_000\Untitled-1.jsonc}
    2 .log        {C:\Users\cppmo_000\debug.log, C:\Users\c…
    2 .md         {C:\Users\cppmo_000\Today.md, C:\Users\cp…
    1 .png        {C:\Users\cppmo_000\g1016.png}
    1 .ps1        {C:\Users\cppmo_000\mini-error.ps1}
    1 .sql        {C:\Users\cppmo_000\foo.sql}
    5 .txt        {C:\Users\cppmo_000\last.txt, C:\Users\cp…



```ps1
# this does not render great, so here's it in markdown
Pwsh🐒> gcm ls | help -online
```
this opens [docs.microsoft.com/../microsoft.powershell.management/get-childitem?view=powershell-7.3&WT.mc_id=ps-gethelp](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.management/get-childitem?view=powershell-7.3&WT.mc_id=ps-gethelp)
or

```ps1
Pwsh🐒> gcm -m ClassExplorer | at -1 | help -online
```
This opens the page: <https://github.com/SeeminglyScience/ClassExplorer/blob/master/docs/en-US/Find-Type.md>

File objects default to formatting like this (on my profile)

In [ ]:
fd --changed-within 10minutes --search-path (gi  "$Env:AppData\code\logs")
| gi
| ? Name -NotMatch 'github|jupyter|git'
| sort LastWriteTime



    Directory: C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T100533\exthost1\output_logging
_20220629T100541

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
-a---           6/29/2022  6:02 PM           1578 26-.NET Interactive  diagnostics.log
-a---           6/29/2022  6:09 PM           4330 11-JSON Language Server.log

    Directory: C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T100533\exthost2\output_logging
_20220629T100541

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
-a---           6/29/2022  6:10 PM           6020 15-JSON Language Server.log



In [ ]:
pushd (gi  "$Env:AppData\code\logs")

fd --changed-within 10minutes --search-path (gi  "$Env:AppData\code\logs")
| gi
| sort LastWriteTime
| To->RelativePath $Env:AppData
| select -first 10

20220629T100533\exthost1\output_logging_20220629T100541\3-GitHub Authentication.log
20220629T100533\exthost1\output_logging_20220629T100541\12-GitHub Pull Request.log
20220629T100533\exthost1\output_logging_20220629T100541\17-Python Test Adapter Log.log
20220629T100533\exthost1\output_logging_20220629T100541\22-Jupyter.log
20220629T100533\exthost2\output_logging_20220629T100541\11-Git.log
20220629T100533\exthost1\output_logging_20220629T100541\26-.NET Interactive  diagnostics.log
20220629T100533\exthost1\output_logging_20220629T100541\14-Git.log
20220629T100533\exthost1\output_logging_20220629T100541\11-JSON Language Server.log
20220629T100533\exthost2\output_logging_20220629T100541\15-JSON Language Server.log


In [ ]:
pushd (gi  "$Env:AppData\code\logs")

fd --changed-within 200minutes --search-path (gi  "$Env:AppData\code\logs") 
| gi
| sort LastWriteTime
| To->RelativePath $Env:AppData
| select -first 10

20220629T161054\
20220629T100533\exthost2\output_logging_20220629T100541\9-GitHub Pull Request.log
20220629T100533\exthost1\output_logging_20220629T100541\
20220629T100533\exthost1\output_logging_20220629T100541\27-.NET Interactive  logger.log
20220629T100533\exthost1\output_logging_20220629T100541\23-VS IntelliCode.log
20220629T100533\exthost1\output_logging_20220629T100541\25-Python 3.9.13 64-bit (windows store)-languageserver.log
20220629T100533\exthost1\output_logging_20220629T100541\9-Python.log
20220629T100533\exthost1\output_logging_20220629T100541\24-Pylint.log
20220629T100533\sharedprocess.log
20220629T165413\
20220629T161054\
20220629T100533\exthost2\output_logging_20220629T100541\9-GitHub Pull Request.log
20220629T100533\exthost1\output_logging_20220629T100541\
20220629T100533\exthost1\output_logging_20220629T100541\27-.NET Interactive  logger.log
20220629T100533\exthost1\output_logging_20220629T100541\23-VS IntelliCode.log
20220629T100533\exthost1\output_logging_20220629T10

In [ ]:
pushd (gi  "$Env:AppData\code\logs")

fd --changed-within 10minutes --search-path (gi  "$Env:AppData\code\logs")
| gi
| ? Name -NotMatch 'github|jupyter|git'
| sort LastWriteTime
| To->RelativePath $Env:AppData

20220629T100533\exthost1\output_logging_20220629T100541\11-JSON Language Server.log
20220629T100533\exthost2\output_logging_20220629T100541\15-JSON Language Server.log


In [ ]:
# slightly different route, preserving colors
fd --changed-within 200minutes --search-path (gi  "$Env:AppData\code\logs") --color=always
| sort { $_ | StripAnsi | Gi | % LastWriteTime }
| select -first 10

C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T161054\
C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T100533\exthost2\output_logging_20220629T100541\9-GitHub Pull Request.log
C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T100533\exthost1\output_logging_20220629T100541\
C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T100533\exthost1\output_logging_20220629T100541\27-.NET Interactive  logger.log
C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T100533\exthost1\output_logging_20220629T100541\23-VS IntelliCode.log
C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T100533\exthost1\output_logging_20220629T100541\25-Python 3.9.13 64-bit (windows store)-languageserver.log
C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T100533\exthost1\output_logging_20220629T100541\9-Python.log
C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T100533\exthost1\output_logging_20220629T100541\24-Pylint.log
C:\Users\cppmo_000\AppData\Roaming\Code\logs\20220629T100533\sha